# Canadian Freight Analysis Framework analysis

In [34]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Read the data

In [7]:
cfaf = pd.read_csv('../../../Canadian Freight Analysis Framework/CFAF_C2011-2017_Description_E.csv')
# filter: Year = 2017 (latest available data), Mode = Truck for-hire
cfaf = cfaf.query('Year==2017 and Mode=="Truck for-hire"')

# Fix columns floats as strings
for col in ['Shipments', 'Weightkg', 'Revenue', 'Distance', 'TonneKm', 'ShipmentValue']:
    cfaf[col] = cfaf[col].str.replace(',', '').astype(float)

# Calculate average shipment distance (km) for an origin-destination (OD) pair
cfaf = cfaf.assign(AvgShipmentDistance = lambda x: x.Distance/x.Shipments)

# Calculate average shipment weight (kg) for an origin-destination (OD) pair
cfaf = cfaf.assign(AvgShipmentWeight = lambda x: x.Weightkg/x.Shipments)

In [8]:
cfaf

,Year,Mode,Commodity,OrigCMA,OrigProv,OrigCtry,DestCMA,DestProv,DestCtry,Shipments,Weightkg,Revenue,Distance,TonneKm,ShipmentValue,AvgShipmentDistance,AvgShipmentWeight
53056,2017,Truck for-hire,Agricultural products,Calgary,Alberta,CANADA,Calgary,Alberta,CANADA,2494.4,44030905.9,2255951.2,52216.9,878949.0,44346784.0,20.933651,17651.902622
53057,2017,Truck for-hire,Agricultural products,Calgary,Alberta,CANADA,Edmonton,Alberta,CANADA,3517.0,8089836.3,1297881.5,1086135.4,2524033.8,14420925.7,308.824396,2300.209355
53058,2017,Truck for-hire,Agricultural products,Calgary,Alberta,CANADA,Halifax,Nova Scotia,CANADA,9.0,4621.3,2320.5,44367.0,22772.9,4902.4,4929.666667,513.477778
53059,2017,Truck for-hire,Agricultural products,Calgary,Alberta,CANADA,Hamilton,Ontario,CANADA,2.0,533.4,1339.7,6916.0,1844.2,775.8,3458.000000,266.700000
53060,2017,Truck for-hire,Agricultural products,Calgary,Alberta,CANADA,Montreal,Quebec,CANADA,23.0,30234.7,14249.3,84688.0,111618.6,35698.4,3682.086957,1314.552174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58716,2017,Truck for-hire,Waste and Scrap,Winnipeg,Manitoba,CANADA,United States and Mexico,United States and Mexico,United States and Mexico,191.0,2960034.2,238871.7,327724.5,4586519.7,6752917.9,1715.835079,15497.561257
58717,2017,Truck for-hire,Waste and Scrap,Winnipeg,Manitoba,CANADA,Vancouver,British Columbia,CANADA,33.0,244308.0,43442.4,75642.0,560857.6,95503.0,2292.181818,7403.272727
58718,2017,Truck for-hire,Waste and Scrap,Winnipeg,Manitoba,CANADA,Winnipeg,Manitoba,CANADA,1684.1,50233479.6,563375.4,29425.7,790471.0,34449328.1,17.472656,29828.085981
58719,2017,Truck for-hire,Waste and Scrap,Winnipeg,Manitoba,CANADA,Yukon,Yukon,CANADA,1.0,786.1,176.6,2901.0,2280.4,307.6,2901.000000,786.100000


### Calculate statistics for each province and long haul distance threshold

In [50]:
province_abbrev = {
    'Alberta': 'AB',
    'British Columbia': 'BC',
    'Manitoba': 'MB',
    'New Brunswick': 'NB',
    'Newfoundland and Labrador': 'NL',
    'Nova Scotia': 'NS',
    'Ontario': 'ON',
    'Prince Edward Island': 'PE',
    'Quebec': 'QC',
    'Saskatchewan': 'SK',
    'Northwest Territories': 'NT',
    'Nunavut': 'NU',
    'Yukon': 'YT'
}

df = pd.DataFrame(list(province_abbrev.items()), columns=['Province', 'region'])
long_haul_threshold = np.arange(100, 1001, 100)
df = df.merge(pd.DataFrame({'long_haul_threshold': long_haul_threshold}), how='cross')

In [51]:
long_haul_threshold = 600    # 600 km: 2x of 300 km operating radius from Sebastian's study

In [52]:
def province_longhaul_stats(Province, long_haul_threshold):
        # Filter for province as origin or destination
        cfaf_prov = cfaf.query('(OrigProv==@Province or DestProv==@Province)')
        # All shipments and tonne-km
        stats_all = cfaf_prov[['Shipments', 'TonneKm']].sum()
        # Long-haul only
        cfaf_prov_longhaul = cfaf_prov.query('AvgShipmentDistance >= @long_haul_threshold')
        stats_longhaul = cfaf_prov_longhaul[['Shipments', 'TonneKm']].sum()
        # Proportions
        prop_shipments = stats_longhaul['Shipments'] / stats_all['Shipments']
        prop_tonnekms = stats_longhaul['TonneKm'] / stats_all['TonneKm']
        # Mean shipment weight (long-haul only)
        mean_weight = cfaf_prov_longhaul['Weightkg'].sum() / cfaf_prov_longhaul['Shipments'].sum()  # average weight of long-haul shipments

        return prop_shipments, prop_tonnekms, mean_weight

In [53]:
df[['prop_shipments_abvthreshold', 'prop_tonnekms_abvthreshold', 'mean_weight_abvthreshold']] = df.apply(
    lambda row: province_longhaul_stats(row['Province'], row['long_haul_threshold']),
    axis=1, result_type='expand'
)

In [54]:
df.query('region=="ON"').sort_values('long_haul_threshold', ascending=False)

,Province,region,long_haul_threshold,prop_shipments_abvthreshold,prop_tonnekms_abvthreshold,mean_weight_abvthreshold
69,Ontario,ON,1000,0.233979,0.572962,6641.944136
68,Ontario,ON,900,0.246674,0.596245,6838.250220
67,Ontario,ON,800,0.276783,0.649522,7415.740267
66,Ontario,ON,700,0.314147,0.685785,7474.947346
65,Ontario,ON,600,0.355940,0.758655,8181.374114
64,Ontario,ON,500,0.453392,0.824903,7712.428234
63,Ontario,ON,400,0.475300,0.839767,7721.408419
62,Ontario,ON,300,0.571864,0.880038,7496.144368
61,Ontario,ON,200,0.690914,0.938152,8046.223261
60,Ontario,ON,100,0.739133,0.969115,8974.781625


In [57]:
df.to_csv('cfaf_province_longhaul_threshold_stats.csv', index=False)

### What % of truck activity in Ontario is long-haul (> 600 km)?

- 36% of truck shipments (aka trucks) are for OD pairs above 600 km
- 76% of truck activity (tonne-km) are for OD pairs above 600 km
- 8.2 tonnes average shipment weight

In [56]:
df.query('region=="ON" and long_haul_threshold==600')

,Province,region,long_haul_threshold,prop_shipments_abvthreshold,prop_tonnekms_abvthreshold,mean_weight_abvthreshold
65,Ontario,ON,600,0.35594,0.758655,8181.374114
